# Neural Network with Keras using Seeds Data

This exercise is taken and modified from https://github.com/benjaminwilson/python-clustering-exercises

This is a class to choose a good number of clusters for a dataset using the k-means inertia graph.  You are given a dataset of the measurements of samples of grain.  What's a good number of clusters in this case?

This dataset was obtained from the [UCI](https://archive.ics.uci.edu/ml/datasets/seeds).


**Step 1:** Load the dataset _(written for you)_.

In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris

In [7]:
# Loading my datasets
seeds_df = pd.read_csv('/Users/setel/Downloads//Data/seeds.csv')

In [8]:
seeds_df #There are 7 attributes to determine the 3 type of grain

,area,perimeter,compactness,length,width,asymmetry_coefficient,groove_length,grain_variety
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,Kama wheat
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,Kama wheat
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,Kama wheat
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,Kama wheat
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,Kama wheat
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,Canadian wheat
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,Canadian wheat
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,Canadian wheat
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,Canadian wheat


In [9]:
dataset = seeds_df.values

In [10]:
dataset

array([[15.26, 14.84, 0.871, ..., 2.221, 5.22, 'Kama wheat'],
       [14.88, 14.57, 0.8811, ..., 1.018, 4.956, 'Kama wheat'],
       [14.29, 14.09, 0.905, ..., 2.699, 4.825, 'Kama wheat'],
       ...,
       [13.2, 13.66, 0.8883, ..., 8.315, 5.056, 'Canadian wheat'],
       [11.84, 13.21, 0.8521, ..., 3.5980000000000003, 5.044,
        'Canadian wheat'],
       [12.3, 13.34, 0.8684, ..., 5.6370000000000005, 5.063,
        'Canadian wheat']], dtype=object)

In [11]:
# X captures all the attributes except the 7th column
X = dataset[:,0:7].astype(float)

# The 7th column is what we want to achieve/output
Y = dataset[:,7]

In [12]:
X

array([[15.26  , 14.84  ,  0.871 , ...,  3.312 ,  2.221 ,  5.22  ],
       [14.88  , 14.57  ,  0.8811, ...,  3.333 ,  1.018 ,  4.956 ],
       [14.29  , 14.09  ,  0.905 , ...,  3.337 ,  2.699 ,  4.825 ],
       ...,
       [13.2   , 13.66  ,  0.8883, ...,  3.232 ,  8.315 ,  5.056 ],
       [11.84  , 13.21  ,  0.8521, ...,  2.836 ,  3.598 ,  5.044 ],
       [12.3   , 13.34  ,  0.8684, ...,  2.974 ,  5.637 ,  5.063 ]])

In [13]:
Y

array(['Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama wheat',
       'Kama wheat', 'Kama wheat', 'Kama wheat', 'Kama 

In [14]:
dataset.shape # 210 rows, 8 columns

(210, 8)

In [15]:
# Now we want to convert these string output to Boolean
# Encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# Convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [16]:
dummy_y

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1

In [17]:
# Now we will create the Neutral Network Model
# Define baseline model
def baseline_model():
	# Create model
	model = Sequential()
	model.add(Dense(8, input_dim=7, activation='relu')) # 8 hidden nodes, with 7 inputs
	model.add(Dense(3, activation='softmax')) # 3 outputs
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [18]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

/var/folders/ks/l2fhkrzd6f1fd06j0ptvfv600000gn/T/ipykernel_27316/216824370.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)


In [19]:
kfold = KFold(n_splits=10, shuffle=True)

In [20]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

2022-02-05 13:53:42.383570: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 90.00% (5.81%)


In [22]:
# What if i were to play around with NN model
# Now we will create the Neutral Network Model
# Define baseline model
def baseline_model_iman():
	# Create model
	model = Sequential()
	model.add(Dense(14, input_dim=7, activation='relu')) # 14 hidden nodes, with 7 inputs
	model.add(Dense(3, activation='softmax')) # 3 outputs
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [23]:
estimator = KerasClassifier(build_fn=baseline_model_iman, epochs=200, batch_size=5, verbose=0)

/var/folders/ks/l2fhkrzd6f1fd06j0ptvfv600000gn/T/ipykernel_27316/2743208476.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model_iman, epochs=200, batch_size=5, verbose=0)


In [24]:
kfold = KFold(n_splits=10, shuffle=True)

In [25]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

In [26]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 91.43% (3.56%)
